# Testing the Creation of Triangloids in SPICE-3
This notebook was used to both plot the output of spice-3 triangloid tests and to recreate teh functionality of the check_halfplane function used to to make them in spice-3's fortran source. 

In [1]:
%matplotlib tk
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
import scipy.io as sio
import sys
import re
import os
import glob
import copy
import pathlib as pth
import importlib
sys.path.append('/home/jleland/Coding/Projects/flopter')
import flopter.spice.splopter as spl
import flopter.spice.tdata as td
import flopter.spice.inputparser as inp
import flopter.core.ivdata as iv
import flopter.core.fitters as fts
import flopter.core.fitdata as fd
import flopter.core.lputils as lpu
import flopter.core.constants as c

In [284]:
# lowdens_dir = pth.Path('/home/jleland/Spice/spice2/bin/data_local_m2/lowdens_anglescan')
# lowdens_dir = pth.Path('/home/jleland/data/external/spice/test/data')
# lowdens_dir = pth.Path('/home/jleland/data/external/spice/test/flush_test')
# lowdens_dir = pth.Path('/home/jleland/data/external/spice/test/successful_flush')
test_dir = pth.Path('/home/jleland/data/external/spice/test/triangles')
os.chdir(test_dir)

In [285]:
test_run_names = list(test_dir.glob('*'))
for i, trn in enumerate(test_run_names):
    print(i, trn)

0 /home/jleland/data/external/spice/test/triangles/triangle_9_srcchange
1 /home/jleland/data/external/spice/test/triangles/triangle_8_noblocks
2 /home/jleland/data/external/spice/test/triangles/triangle_7
3 /home/jleland/data/external/spice/test/triangles/triangle_6
4 /home/jleland/data/external/spice/test/triangles/triangle_5
5 /home/jleland/data/external/spice/test/triangles/triangle_4
6 /home/jleland/data/external/spice/test/triangles/triangle_3
7 /home/jleland/data/external/spice/test/triangles/triangle_2
8 /home/jleland/data/external/spice/test/triangles/triangle_18_angletest_full
9 /home/jleland/data/external/spice/test/triangles/triangle_17_angletest4
10 /home/jleland/data/external/spice/test/triangles/triangle_16_angletest3
11 /home/jleland/data/external/spice/test/triangles/triangle_15_angletest2
12 /home/jleland/data/external/spice/test/triangles/triangle_14_angletest
13 /home/jleland/data/external/spice/test/triangles/triangle_13_srcchangeback
14 /home/jleland/data/external/

In [287]:
lowdens_dir = test_run_names[8]
lowdens_dir

PosixPath('/home/jleland/data/external/spice/test/triangles/triangle_18_angletest_full')

In [288]:
non_standard_variables = {'objects', 'objectsenum', 'edges', 'Pot'}
desired_variables = td.DEFAULT_REDUCED_DATASET | non_standard_variables

In [289]:
for item in lowdens_dir.glob('*'):
    print(item)

/home/jleland/data/external/spice/test/triangles/triangle_18_angletest_full/test-t.mat
/home/jleland/data/external/spice/test/triangles/triangle_18_angletest_full/test-tT00.mat
/home/jleland/data/external/spice/test/triangles/triangle_18_angletest_full/test-triangle.inp
/home/jleland/data/external/spice/test/triangles/triangle_18_angletest_full/test-0.log


In [290]:
datafile_oi = 'test-t.mat'
tTfile_oi = 'test-tT00.mat'

# tdata = sio.loadmat(datafile_oi, variable_names=non_standard_variables)
tdata = sio.loadmat(lowdens_dir / datafile_oi)
tTdata = sio.loadmat(lowdens_dir / tTfile_oi)

In [291]:
print(tdata.keys())
print(tTdata.keys())
print(tTdata['alphaxz'])
print(tTdata['alphayz'])

dict_keys(['__header__', '__version__', '__globals__', 'version', 'Nx', 'Ny', 'Nz', 'nospecies', 'procmax', 'historyntimes', 'count', 'Na', 'Np', 'Nc', 'Pot', 'Potvac', 'Ex', 'Ey', 'Ez', 'objects', 'edges', 'edgecharge', 'bcmatrix', 'snumber', 'objectscurrent', 'rho01', 'vx01', 'vy01', 'vz01', 'rho02', 'vx02', 'vy02', 'vz02'])
dict_keys(['__header__', '__version__', '__globals__', 'version', 'bstep', 'floatconstant', 'irelhistory', 'objectpot', 'vact', 'nospecies', 'procmax', 'nobjects', 'historyntimes', 'nodiagslots', 'Np', 'Na', 'Nc', 'count', 'Nx', 'Ny', 'Nz', 'dx', 'dy', 'dz', 'dt', 'ksi', 'tau', 'mu', 'P0', 'PL', 'alphaxz', 'alphayz', 'poisslevels', 'poisssmooths', 'poissres', 'mksB', 'mksn0', 'mksTe', 'mksmainionm', 'mksmainionq', 'mkspar1', 'mkspar2', 'mkspar3', 'Lxstart', 'Lystart', 'Nxstart', 'Nystart', 'Nzstart', 'Nxstop', 'Nystop', 'Nzstop', 'Nxtotal', 'Nytotal', 'Nztotal', 'snumber', 'Pot', 'Pottotal', 'edgechargetotal', 'rhotottotal', 'Potvac', 'Ex', 'Ey', 'Ez', 'Ts', 'inj

In [292]:
print(tdata['edges'].shape)

objectsenum = (tdata['objects'] > 0).astype(np.int64)
print(objectsenum.shape)

(65, 65, 65)
(65, 65, 65)


In [293]:
plt.figure()
plt.pcolormesh(objectsenum.sum(axis=2))
plt.colorbar()

In [298]:
fig, ax = plt.subplots(2, figsize=[5,10])

ax[0].pcolormesh(np.transpose(objectsenum[32,:,:]))
ax[1].pcolormesh(np.transpose(tdata['Pot'][32,:,:]))
for i in range(2):
    z_ext = 15
    ax[1].quiver(56, 56, 1, 1 * np.tan(tTdata['alphayz']), scale=z_ext)


In [297]:
fig, ax = plt.subplots()

print(tdata['objectscurrent'][0][0].shape)
ax.plot(tdata['objectscurrent'][0][0])

(31841,)


In [161]:
def plot_slices_3d(arr, z=59, x=32, y=32, axes=None):
    if axes is None:
        fig, axes = plt.subplots(2, 2)
    else:
        fig = axes[0,0].figure
    
    im = axes[0][0].pcolormesh(arr[:,:,z])
    fig.colorbar(im, ax=axes[0][0])
    axes[0][0].set_title(f'Section at z={z}')
    axes[0][0].axvline(x=y, **c.AX_LINE_DEFAULTS)
    axes[0][0].axhline(y=x, **c.AX_LINE_DEFAULTS)
    
    im = axes[1][0].pcolormesh(np.flip(np.transpose(arr[x])))
    fig.colorbar(im, ax=axes[1][0])
    axes[1][0].set_title(f'Section at x={x}')
    axes[1][0].axvline(x=x, **c.AX_LINE_DEFAULTS)
    axes[1][0].axhline(y=arr.shape[0]-z, **c.AX_LINE_DEFAULTS)
    
    im = axes[0][1].pcolormesh(arr[:,y])
    fig.colorbar(im, ax=axes[0][1])
    axes[0][1].set_title(f'Section at y={y}')
    axes[0][1].axvline(x=z, **c.AX_LINE_DEFAULTS)
    axes[0][1].axhline(y=y, **c.AX_LINE_DEFAULTS)

    return axes


# plot_slices_3d(tdata['objects'], x=256)
# plot_slices_3d(objectsenum, x=256)
fig, ax = plt.subplots(2, 2)
plot_slices_3d(objectsenum, x=23, z=31, y=30, axes=ax)


array([[<matplotlib.axes._subplots.AxesSubplot object at 0x7f5baf56f9b0>,
      dtype=object)

In [126]:
plot_slices_3d(tdata['Pot'], x=256, y=256)

IndexError: index 256 is out of bounds for axis 0 with size 65

In [127]:
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure()
ax = fig.gca(projection='3d')

X = np.arange(objectsenum.shape[0])
Y = np.arange(objectsenum.shape[1])
X, Y = np.meshgrid(X, Y)
print(len(X))

ax.plot_surface(X, Y, objectsenum.sum(axis=2))



65


In [128]:
fig = plt.figure()
ax = fig.gca(projection='3d')

xp = []
yp = []
zp = []

for i in range(objectsenum.shape[0]):
    for j in range(objectsenum.shape[1]):
        for k in range(objectsenum.shape[2]):
            if objectsenum[i,j,k]:
                xp.append(i)
                yp.append(j)
                zp.append(k)

ax.scatter(xp, yp, zp)

In [133]:
fig = plt.figure()
ax = fig.gca(projection='3d')

ax.voxels(objectsenum)

{(0, 0, 0): <mpl_toolkits.mplot3d.art3d.Poly3DCollection at 0x7f5bb64dca58>,
 (0, 0, 2): <mpl_toolkits.mplot3d.art3d.Poly3DCollection at 0x7f5bb64eeeb8>,
 (0, 1, 0): <mpl_toolkits.mplot3d.art3d.Poly3DCollection at 0x7f5bb64eec88>,
 (0, 1, 2): <mpl_toolkits.mplot3d.art3d.Poly3DCollection at 0x7f5bb64eea58>,
 (0, 2, 0): <mpl_toolkits.mplot3d.art3d.Poly3DCollection at 0x7f5bb64ee828>,
 (0, 2, 2): <mpl_toolkits.mplot3d.art3d.Poly3DCollection at 0x7f5bb64ee5f8>,
 (0, 3, 0): <mpl_toolkits.mplot3d.art3d.Poly3DCollection at 0x7f5bb64ee3c8>,
 (0, 3, 2): <mpl_toolkits.mplot3d.art3d.Poly3DCollection at 0x7f5bb64ee198>,
 (0, 4, 0): <mpl_toolkits.mplot3d.art3d.Poly3DCollection at 0x7f5bb6604f28>,
 (0, 4, 2): <mpl_toolkits.mplot3d.art3d.Poly3DCollection at 0x7f5bb64f40f0>,
 (0, 5, 0): <mpl_toolkits.mplot3d.art3d.Poly3DCollection at 0x7f5bb64f4240>,
 (0, 5, 2): <mpl_toolkits.mplot3d.art3d.Poly3DCollection at 0x7f5bb654fc88>,
 (0, 6, 0): <mpl_toolkits.mplot3d.art3d.Poly3DCollection at 0x7f5bb65a4630>,

## Checking the functionality of Triangle Drawing and the check_halfplane Function

In [17]:
import math

def check_halfplane(ya, za, yb, zb, yc, zc, y1, z1):
    if zb != za:
        a = (yb - ya)/(zb - za)
        b = - (yb - ya)/(zb - za) * za + ya
        bc = -a*zc + yc
        yt = a*z1 + b
        ytc = a*z1 + bc
        
        if (np.abs(yt - y1) <= np.abs(yt - ytc) and ((yt >= y1 and yt >= ytc) or (yt <= y1 and yt <= ytc))):
            return 1
        else:
            return 0
    else:
        if math.copysign(1.0, zc - za) == math.copysign(1.0, z1 - za) and np.abs(zc - za) >= np.abs(z1 - za):
            return 1
        else:
            return 0

In [80]:
# the working z-plane triangle
ya = 45
yb = 63
yc = 45

za = 31
zb = 31
zc = 31

xa = 20
xb = 20
xc = 44

In [81]:
is_in_triangle = np.zeros([65, 65, 3])

In [82]:
is_in_triangle[20,0,0] = 10

In [83]:
for ix in range(is_in_triangle.shape[0]):
    for iy in range(is_in_triangle.shape[0]):
        
        # z-plane
        is_in_triangle[iy,ix,0] = check_halfplane(xa,ya,xb,yb,xc,yc,ix,iy)
        is_in_triangle[iy,ix,1] = check_halfplane(xc,yc,xa,ya,xb,yb,ix,iy)
        is_in_triangle[iy,ix,2] = check_halfplane(xb,yb,xc,yc,xa,ya,ix,iy)
#         is_in_triangle[iy,ix,0] = check_halfplane(ya,xa,yb,xb,yc,xc,iy,ix)
#         is_in_triangle[iy,ix,1] = check_halfplane(yc,xc,ya,xa,yb,xb,iy,ix)
#         is_in_triangle[iy,ix,2] = check_halfplane(yb,xb,yc,xc,ya,xa,iy,ix)    



In [84]:
fig, ax = plt.subplots(3, figsize=[5,9])

for i in range(3):
    ax[i].pcolormesh(is_in_triangle[:,:,i],)
    ax[i].plot(xa, ya, 'o', xb, yb, 'o', xc, yc, 'o', color='w')



In [58]:
fig, ax = plt.subplots()

ax.pcolormesh(np.logical_and.reduce(is_in_triangle, axis=2))
ax.plot(xa, ya, 'o', xb, yb, 'o', xc, yc, 'o', color='w')

In [59]:
# the non-working x-plane triangle
ya = 0
yb = 0
yc = 19

xa = 20
xb = 20
xc = 20

za = 31
# zb = 31 <-- this one was incorrectly set
zb = 40
zc = 40

In [60]:
is_in_triangle = np.zeros([65, 65, 3])

In [65]:
# for ix in range(is_in_triangle.shape[0]):
for iy in range(is_in_triangle.shape[0]):
    for iz in range(is_in_triangle.shape[0]):
        # x-plane (orientation = 1)
        is_in_triangle[iz,iy,0] = check_halfplane(ya,za,yb,zb,yc,zc,iy,iz)
        is_in_triangle[iz,iy,1] = check_halfplane(yc,zc,ya,za,yb,zb,iy,iz)
        is_in_triangle[iz,iy,2] = check_halfplane(yb,zb,yc,zc,ya,za,iy,iz)
#         is_in_triangle[iz,iy,0] = check_halfplane(za,ya,zb,yb,zc,yc,iz,iy)
#         is_in_triangle[iz,iy,1] = check_halfplane(zc,yc,za,ya,zb,yb,iz,iy)
#         is_in_triangle[iz,iy,2] = check_halfplane(zb,yb,zc,yc,za,ya,iz,iy)


In [66]:
fig, ax = plt.subplots(3, figsize=[5,9])

for i in range(3):
    ax[i].pcolormesh(is_in_triangle[:,:,i],)
    ax[i].plot(ya, za, 'o', yb, zb, 'o', yc, zc, 'o', color='w')


In [47]:
fig, ax = plt.subplots()

ax.pcolormesh(np.logical_and.reduce(is_in_triangle, axis=2))
ax.plot(ya, za, 'o', yb, zb, 'o', yc, zc, 'o', color='w')

In [109]:
# y-plane triangle! (orientation = 2)
xa = 20
xb = 44 
xc = 44

ya = 44
yb = 44
yc = 44

za = 31
zb = 31
zc = 40

In [113]:
is_in_triangle = np.zeros([65, 65, 3])
for ix in range(is_in_triangle.shape[0]):
    for iz in range(is_in_triangle.shape[0]):
        is_in_triangle[iz,ix,0] = check_halfplane(xa,za,xb,zb,xc,zc,ix,iz)
        is_in_triangle[iz,ix,1] = check_halfplane(xc,zc,xa,za,xb,zb,ix,iz)
        is_in_triangle[iz,ix,2] = check_halfplane(xb,zb,xc,zc,xa,za,ix,iz)
#         is_in_triangle[iz,ix,0] = check_halfplane(za,xa,zb,xb,zc,xc,iz,ix)
#         is_in_triangle[iz,ix,1] = check_halfplane(zc,xc,za,xa,zb,xb,iz,ix)
#         is_in_triangle[iz,ix,2] = check_halfplane(zb,xb,zc,xc,za,xa,iz,ix)


In [114]:
fig, ax = plt.subplots(3, sharex=True, sharey=True, figsize=[5,9])

for i in range(3):
    ax[i].pcolormesh(is_in_triangle[:,:,i],)
    ax[i].plot(xa, za, 'o', xb, zb, 'o', xc, zc, 'o', color='w')
    
ax[2].set_xlabel('x')
ax[0].set_ylabel('z')
ax[1].set_ylabel('z')
ax[2].set_ylabel('z')


Text(0, 0.5, 'z')

In [115]:
fig, ax = plt.subplots()

ax.pcolormesh(np.logical_and.reduce(is_in_triangle, axis=2))
ax.plot(xa, za, 'o', xb, zb, 'o', xc, zc, 'o', color='w')
ax.set_xlabel('x')
ax.set_ylabel('z')

Text(0, 0.5, 'z')